In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, GlobalAveragePooling2D,Dropout,Conv2D
from keras.applications.densenet import DenseNet121
from keras.models import Model
import keras.backend as K
from sklearn.metrics import roc_auc_score
import cv2
import os
import random
from sklearn.utils import shuffle
from keras.applications.mobilenetv2 import MobileNetV2
from keras.callbacks import Callback,ModelCheckpoint
import pydicom
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.losses import binary_crossentropy
from keras.utils import Sequence
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
from tqdm import tqdm
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

class Metrics(Callback):
    def on_train_begin(self, logs={}):
     self.val_f1s = []
     self.val_recalls = []
     self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
     val_predict = (np.asarray(self.model.predict(X_test_feats))).round()
     val_targ = Y_test
#      print(val_targ.shape)   
#      print(val_targ)
#      print(val_predict.shape)   
#      print(val_predict)   
     targ = []
     pred = []
     for x in val_targ:
        targ.append(np.argmax(x))
     for x in val_predict:
        pred.append(np.argmax(x))
#      print(targ)
#      print(pred)
        
     _val_f1 = f1_score(targ, pred)
     
     _val_recall = recall_score(targ, pred)
     _val_precision = precision_score(targ, pred)
     self.val_f1s.append(_val_f1)
     self.val_recalls.append(_val_recall)
     self.val_precisions.append(_val_precision)
     print("— val_f1: {0} — val_precision: {1} — val_recall {2} ".format(_val_f1, _val_precision, _val_recall))
     return

metrics = Metrics()

In [3]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class roc_callback(Callback):
    def __init__(self,validation_data):
#         self.x = training_data[0]
#         self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]


    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
#         y_pred = self.model.predict(self.x)
#         roc = roc_auc_score(self.y, y_pred)
        if(epoch%1==0):
            print("Calc Roc")
            all_rocs = []
            y_pred_val = self.model.predict(X_test,verbose=1)
            for d in range(15):
                try:
                    roc_val = roc_auc_score(Y_test[:,d], y_pred_val[:,d])
                    all_rocs.append(roc_val)
                except:
                    pass
            all_rocs = np.array(all_rocs)
            mean_roc = np.mean(all_rocs)
            
            f1s = {}
            for d in range(15):
                try:
                    score = f1_score(Y_test[:,d], np.around(y_pred_val[:,d]))
                    f1s[global_cls[d]] = score
                except:
                    print(global_cls[d])
            avg_f1 = np.average(list(f1s.values()))
            
            print("ROC VAL {0}".format(mean_roc))
            print("AVG F1 {0}".format(avg_f1))
        return

    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [5]:
roc_call =roc_callback((X_test,Y_test))

In [6]:
global_cls = []
global_cls = np.array(global_cls)

# Model 1: DenseNet121

In [7]:
def unweighted_binary_crossentropy(y_true, y_pred):
    return K.mean(K.binary_crossentropy(y_true, y_pred))

In [8]:
dense_model = DenseNet121(weights='imagenet', include_top=False,input_shape=(224,224,3),pooling='avg')

In [9]:
preds = Dense(14,activation='sigmoid')(dense_model.output)
model = Model(dense_model.input,preds)
model.compile(optimizer='rmsprop',loss=unweighted_binary_crossentropy,metrics=['accuracy'])
model.load_weights('weights.hdf5')

#pad_model = DenseNet121

#Approx F1 : 35-36

In [10]:
new_preds = Dense(15,activation='sigmoid')(dense_model.output)
pad_model = Model(dense_model.input,new_preds)
pad_model.compile(optimizer='rmsprop',loss=unweighted_binary_crossentropy,metrics=['accuracy'])

In [12]:
model_checkpoint = ModelCheckpoint(
        os.path.join('./', 'DenseNet.hdf5'),
        monitor='val_acc', mode='max',save_best_only=True, verbose=1)
callbacks = [model_checkpoint, roc_call]

In [ ]:
pad_model.fit(X_train,Y_train,validation_data=(X_test,Y_test), epochs=50, batch_size = 32, callbacks=callbacks)

Train on 13118 samples, validate on 12815 samples
Epoch 1/50
13118/13118 [==============================] - 488s 37ms/step - loss: 0.1322 - acc: 0.1600 - val_loss: 0.1436 - val_acc: 0.1205

Epoch 00001: val_acc improved from -inf to 0.12048, saving model to ./DenseNet.hdf5
Calc Roc
12815/12815 [==============================] - 111s 9ms/step
ROC VAL 0.912570202205982
AVG F1 0.17125509194651165
Epoch 2/50


/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


13118/13118 [==============================] - 451s 34ms/step - loss: 0.1023 - acc: 0.1971 - val_loss: 0.1364 - val_acc: 0.1383

Epoch 00002: val_acc improved from 0.12048 to 0.13828, saving model to ./DenseNet.hdf5
Calc Roc
12815/12815 [==============================] - 108s 8ms/step
ROC VAL 0.9214705717303991
AVG F1 0.27097151511540946
Epoch 3/50
13118/13118 [==============================] - 451s 34ms/step - loss: 0.0878 - acc: 0.2202 - val_loss: 0.1833 - val_acc: 0.1413

Epoch 00003: val_acc improved from 0.13828 to 0.14132, saving model to ./DenseNet.hdf5
Calc Roc
12815/12815 [==============================] - 107s 8ms/step
ROC VAL 0.9098418936034529
AVG F1 0.2765162898343201
Epoch 4/50
13118/13118 [==============================] - 450s 34ms/step - loss: 0.0746 - acc: 0.2417 - val_loss: 0.1658 - val_acc: 0.1606

Epoch 00004: val_acc improved from 0.14132 to 0.16059, saving model to ./DenseNet.hdf5
Calc Roc
12815/12815 [==============================] - 108s 8ms/step
ROC VAL 0.920

In [11]:
pad_model.load_weights('./DenseNet.hdf5')

In [12]:
test_preds = pad_model.predict(X_test, verbose = 1)

12815/12815 [==============================] - 113s 9ms/step


# Model 2: MobileNetv2

In [7]:
mobilenet_model = MobileNetV2(weights='imagenet', include_top=False,input_shape=(224,224,3),pooling='avg')
preds = Dense(15,activation='softmax')(mobilenet_model.output)
model = Model(mobilenet_model.input,preds)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model_checkpoint = ModelCheckpoint(
        os.path.join('./', 'mobilenet.hdf5'),
        monitor='val_acc', mode='max',save_best_only=True, verbose=1)
callbacks = [model_checkpoint, roc_call]

In [9]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test), epochs=50, batch_size = 32, callbacks=callbacks)

Train on 13118 samples, validate on 12815 samples
Epoch 1/50
13118/13118 [==============================] - 218s 17ms/step - loss: 1.6846 - acc: 0.1138 - val_loss: 6.7484 - val_acc: 0.0771

Epoch 00001: val_acc improved from -inf to 0.07710, saving model to ./mobilenet.hdf5
Calc Roc
12815/12815 [==============================] - 42s 3ms/step
ROC VAL 0.748363649175161
AVG F1 0.026050908041063022
Epoch 2/50


/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


13118/13118 [==============================] - 206s 16ms/step - loss: 1.5705 - acc: 0.1302 - val_loss: 7.6532 - val_acc: 0.0371

Epoch 00002: val_acc did not improve from 0.07710
Calc Roc
12815/12815 [==============================] - 41s 3ms/step
ROC VAL 0.6210831577707151
AVG F1 0.035024478505006715
Epoch 3/50
13118/13118 [==============================] - 206s 16ms/step - loss: 1.5184 - acc: 0.1437 - val_loss: 6.2247 - val_acc: 0.1020

Epoch 00003: val_acc improved from 0.07710 to 0.10199, saving model to ./mobilenet.hdf5
Calc Roc
12815/12815 [==============================] - 41s 3ms/step
ROC VAL 0.7129307086355929
AVG F1 0.03170762738201333
Epoch 4/50
13118/13118 [==============================] - 206s 16ms/step - loss: 1.4733 - acc: 0.1491 - val_loss: 6.7200 - val_acc: 0.0925

Epoch 00004: val_acc did not improve from 0.10199
Calc Roc
12815/12815 [==============================] - 41s 3ms/step
ROC VAL 0.66897588125275
AVG F1 0.022408282574436396
Epoch 5/50
13118/13118 [==========

KeyboardInterrupt: 

In [7]:
model.load_weights('./mobilenet.hdf5')

In [8]:
test_preds = model.predict(X_test, verbose = 1)

12815/12815 [==============================] - 42s 3ms/step


In [9]:
model.evaluate(X_test,Y_test)

12815/12815 [==============================] - 42s 3ms/step


[4.470160653318598, 0.11111978150604761]

In [16]:
pad_f1s = {}
avg_f1s = []
for d in range(15):
    try:
        score = f1_score(Y_test[:,d], np.around(test_preds[:,d]))
        pad_f1s[global_cls[d]] = score
        avg_f1s.append(score*np.sum(Y_test[:,d]))
    except:
        print(global_cls[d])

a
np.mean(pad_f1s.values())

TypeError: unsupported operand type(s) for /: 'dict_values' and 'int'

# Model 3:

In [20]:
np.mean(list(pad_f1s.values()))

0.36492159242160777